In [0]:
from delta.tables import DeltaTable

In [0]:
data = spark.read.table("example.bronze_db.customers_view_filled")

table_path = "/Volumes/example/bronze_db/customers_partitioned"
data.write.format("delta").mode("overwrite").partitionBy('state').save(table_path)


spark.sql("OPTIMIZE delta.`/Volumes/example/bronze_db/customers_partitioned` ZORDER BY (customer_id);")

deltaTable = DeltaTable.forPath(spark, "/Volumes/example/bronze_db/customers_partitioned")
display(deltaTable.history())


version,timestamp,userId,userName,operation,operationParameters,job,notebook,clusterId,readVersion,isolationLevel,isBlindAppend,operationMetrics,userMetadata,engineInfo
2,2026-01-18T12:49:05.000Z,73331271719467,kalyanmistcse@gmail.com,WRITE,"Map(mode -> Overwrite, statsOnLoad -> false, partitionBy -> [""state""])",null,List(396297392114963),0118-122226-ucqupaow-v2n,1,WriteSerializable,false,"Map(numFiles -> 51, numRemovedFiles -> 51, numRemovedBytes -> 2837427, numDeletionVectorsRemoved -> 0, numOutputRows -> 28813, numOutputBytes -> 2837427)",null,Databricks-Runtime/17.3.x-aarch64-photon-scala2.13
1,2026-01-18T12:46:28.000Z,73331271719467,kalyanmistcse@gmail.com,WRITE,"Map(mode -> Overwrite, statsOnLoad -> false, partitionBy -> [""state""])",null,List(396297392114963),0118-122226-ucqupaow-v2n,0,WriteSerializable,false,"Map(numFiles -> 51, numRemovedFiles -> 51, numRemovedBytes -> 2837427, numDeletionVectorsRemoved -> 0, numOutputRows -> 28813, numOutputBytes -> 2837427)",null,Databricks-Runtime/17.3.x-aarch64-photon-scala2.13
0,2026-01-18T12:31:19.000Z,73331271719467,kalyanmistcse@gmail.com,WRITE,"Map(mode -> Overwrite, statsOnLoad -> false, partitionBy -> [""state""])",null,List(1379506041694369),0118-122226-ucqupaow-v2n,null,WriteSerializable,false,"Map(numFiles -> 51, numRemovedFiles -> 0, numRemovedBytes -> 0, numDeletionVectorsRemoved -> 0, numOutputRows -> 28813, numOutputBytes -> 2837427)",null,Databricks-Runtime/17.3.x-aarch64-photon-scala2.13


In [0]:
# Explain query
spark.sql("SELECT * FROM example.bronze_db.customers_view_filled").explain(True)

# Partitioned table
spark.sql("""
  CREATE TABLE if not EXISTS example.bronze_db.customers_view_filled
  USING DELTA
  PARTITIONED BY (state)
  AS SELECT * FROM example.bronze_db.customers_view_filled
""")

# # Optimize
# spark.sql("OPTIMIZE example.bronze_db.customers_partitioned ZORDER BY (customer_id)")

# Benchmark
import time
start = time.time()
spark.sql("SELECT * FROM example.bronze_db.customers_view_filled WHERE customer_id=12345").count()
print(f"Time: {time.time()-start:.2f}s")

# # Cache for iterative queries
# cached = spark.table("example.bronze_db.customers_view_filled").cache()
# cached.count()  # Materialize


== Parsed Logical Plan ==
'Project [*]
+- 'UnresolvedRelation [example, bronze_db, customers_view_filled], [], false

== Analyzed Logical Plan ==
customer_id: double, tax_id: string, tax_code: string, customer_name: string, state: string, city: string, postcode: string, street: string, number: string, unit: string, region: string, district: string, lon: double, lat: double, ship_to_address: string, valid_from: double, units_purchased: double, loyalty_segment: double
Project [customer_id#31318, tax_id#31319, tax_code#31320, customer_name#31321, state#31322, city#31323, postcode#31324, street#31325, number#31326, unit#31327, region#31328, district#31329, lon#31330, lat#31331, ship_to_address#31332, valid_from#31333, units_purchased#31334, loyalty_segment#31335]
+- SubqueryAlias example.bronze_db.customers_view_filled
   +- View (`example`.`bronze_db`.`customers_view_filled`, [customer_id#31318, tax_id#31319, tax_code#31320, customer_name#31321, state#31322, city#31323, postcode#31324, st